In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

df = pd.read_csv('train.csv').drop(columns=['Id'])
dfPred = pd.read_csv('testX.csv')

In [2]:
def f(x):
    pca = PCA(n_components = 20)
    ss = StandardScaler()
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(pca.fit_transform(ss.fit_transform(df.iloc[:,1:].values)), df.iloc[:,0], test_size=0.2, random_state=x)
    X_pred = pca.transform(ss.transform(dfPred.iloc[:,1:].values))
    clf = SVC(random_state=x, C = 35)
    clf.fit(X_train_pca, y_train)
    return [x, '{0:.2f}%'.format(clf.score(X_test_pca, y_test)*100), clf.predict(X_pred)]

result = []
with Pool(40) as p:
    result = p.map(f, [*range(40)])

print(result[0])

[0, '88.66%', array([4, 4, 0, ..., 2, 2, 2])]


In [5]:
for i in range(40):
    print(result[i][1])

88.66%
88.52%
88.46%
88.19%
88.50%
88.90%
88.24%
88.92%
88.33%
89.16%
88.50%
88.22%
88.73%
88.48%
87.90%
88.55%
88.53%
88.61%
88.43%
88.67%
88.76%
88.39%
88.25%
88.88%
88.81%
88.96%
88.28%
88.46%
88.54%
88.98%
88.42%
88.87%
88.23%
88.42%
88.35%
89.09%
88.26%
88.52%
88.38%
88.58%


In [3]:
dfRes = dfPred[['Id']].copy()
for i in range(40):
    dfRes[result[i][0]] = result[i][2]
dfRes

,Id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,0,4,4,3,3,3,3,3,4,3,...,4,3,3,3,3,3,3,3,4,3
1,1,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,2,1,1,2,1,1,2,2,1,...,2,1,2,1,2,1,2,2,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,3,3,3,3,3,3,1,3,3,...,3,3,3,3,3,3,3,3,3,3
9996,9996,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9997,9997,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
9998,9998,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [8]:
count = dfRes.apply(pd.Series.value_counts, axis=1)[[0, 1, 2, 3, 4]].fillna(0)

In [11]:
toFile = dfRes.drop(columns=['Id']).mode(axis=1)[[0]].astype(int).rename(columns={0: 'Label'})
toFile.index = toFile.index.rename('Id')
toFile.to_csv('SVMRandomCombine90.csv', encoding='utf-8')